# Overview
In this notebook we explore multithreading using python. We will first discuss relevant libraries before looking at a specific implimentation.

# Relevant Libraries
## The multiprocessing Module
The multiprocessing module is at the heart of python's parallelization capabilities. From this module we gain access to both multithreading and multiprocessing libraries and primitives. Additionally we get access to the synchronization primitives.

Below we outline the key objects required for most implimentations:

### Shared Memory
Similar to the Value and Array objects used in multithreading, the multiprocessing library provides a mechanism for sharing memory between processes. This is in the shared_memory submodule. 

#### SharedMemory
The [SharedMemory object](https://docs.python.org/3/library/multiprocessing.shared_memory.html) provides a memory buffer which can be shared between processes. Each buffer has a unique name which globally identifies it between processes. The buffer needs to be cleaned up manually. If we are cleaning up for a single process we use the close() function. If we want to destory the buffer once all the processes have called close() we call unlink(). If we do not call unlink, the memory will persist and can be accidentally reloaded with an old state!

To create a shared memory buffer we would do the following:

In [6]:
import multiprocessing

# Set a name to uniquely identify the buffer
shared_memory_name = "multiprocessor"

# Create an instance if one does not exist
try:
    shared_memory = multiprocessing.shared_memory.SharedMemory(name=shared_memory_name, create=True, size=1)
except FileExistsError as fee:
    shared_memory = multiprocessing.shared_memory.SharedMemory(name=shared_memory_name, create=False, size=1)

# Set a value in the buffer
shared_memory_buffer = shared_memory.buf
shared_memory_buffer[0] = 0

# Cleanup
shared_memory.close()
shared_memory.unlink()

If we create an array of values we can access them via index:

In [7]:
# Set a name to uniquely identify the buffer
shared_memory_name = "multiprocessor"

# Create an instance if one does not exist
try:
    shared_memory = multiprocessing.shared_memory.SharedMemory(name=shared_memory_name, create=True, size=4)
except FileExistsError as fee:
    shared_memory = multiprocessing.shared_memory.SharedMemory(name=shared_memory_name, create=False, size=4)

# Set a value in the buffer
shared_memory_buffer = shared_memory.buf
shared_memory_buffer[0] = 0
shared_memory_buffer[1] = 1
shared_memory_buffer[2] = 2
shared_memory_buffer[3] = 3

# Cleanup
shared_memory.close()
shared_memory.unlink()

### Manager
The [Manager object](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.sharedctypes.multiprocessing.Manager) can be used for sharing objects between processes. This object will create proxies between the subprocesses to allow sharing between them.

One of the key reasons to use this object to to get access to a Lock() or RLock() object which can be used between processes.

#### Manager.Lock()

In [2]:
import multiprocessing
manager = multiprocessing.Manager()
lock = manager.Lock()

## The multiprocessing.pool Module
This module provides access to the ThreadPool and must be imported to gain access.


### Pool
The [Pool](https://docs.python.org/3/library/multiprocessing.html#module-multiprocessing.pool)  controls a pool of worker processes. The Pool is interface compatable with the ThreadPool object used to manage thread. The Pool provides several mechanisms to running code segments in parallel. We will only cover one method here; the *startmap()* method. Later we will see a non-blocking option.

In [4]:
import multiprocessing
import time

# Create a lock to synchronize access to STDOUT (the console)
manager = multiprocessing.Manager()
lock = manager.Lock()

# Create a function to print to the console
def my_func(idx, lock):
    for i in range(0, 3):
        lock.acquire()
        print("{0} -> {1}".format(idx, i))
        lock.release()
        time.sleep(1)
    return idx
        
# Create a ThreadPool with three workers
my_pool = multiprocessing.Pool(processes=3)

# Create ten sets of parameters for the 10 functions we want to run in parallel
params = [(idx, lock) for idx in range(9)]

# Parallelize the functions, wait until they all complete, and assemble the results in an array coresponding to the input parameter order
apply_result = my_pool.starmap(my_func, params)

# Get the results in an array coresponding to the input parameter order
print(apply_result)

0 -> 0
1 -> 0
2 -> 0
0 -> 1
1 -> 1
2 -> 1
0 -> 2
1 -> 2
2 -> 2
3 -> 0
4 -> 0
5 -> 0
3 -> 1
4 -> 1
5 -> 1
3 -> 2
4 -> 2
5 -> 2
6 -> 0
7 -> 0
8 -> 0
6 -> 1
7 -> 1
8 -> 1
6 -> 2
7 -> 2
8 -> 2
[0, 1, 2, 3, 4, 5, 6, 7, 8]


Process ForkPoolWorker-8:
Process ForkPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-7:
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/local/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/local/lib/python3.9/multiprocessing/process.py", line 108, 

It is important to note that the ThreadPool will halt at the first error. All reqults etc will be lost if an unhandled exception is encountered

In [49]:
import multiprocessing
import time

# Create a lock to synchronize access to STDOUT (the console)
lock = multiprocessing.Lock()

# Create a function to print to the console
def my_func(idx, lock):
    if idx == 4:
        raise Exception("This should have been caught")
    for i in range(0, 3):
        lock.acquire()
        print("{0} -> {1}".format(idx, i))
        lock.release()
        time.sleep(1)
    return idx
        
# Create a ThreadPool with three workers
my_pool = multiprocessing.pool.ThreadPool(processes=3)

# Create ten sets of parameters for the 10 functions we want to run in parallel
params = [(idx, lock) for idx in range(9)]

# Parallelize the functions, wait until they all complete, and assemble the results in an array coresponding to the input parameter order
apply_result = my_pool.starmap(my_func, params)

# Get the results in an array coresponding to the input parameter order
print(apply_result)

0 -> 0
1 -> 0
2 -> 0
0 -> 1
1 -> 1
2 -> 1
0 -> 2
1 -> 2
2 -> 2
3 -> 0
5 -> 0
6 -> 0
3 -> 1
6 -> 1
5 -> 1
3 -> 2
6 -> 2
5 -> 2
7 -> 0
8 -> 0
7 -> 1
8 -> 1
7 -> 2
8 -> 2


Exception: This should have been caught

We can see only one exception was raised and we dont know what parameters were responsible.

To remedy this problem, and others, we will impliment a more robust object.

# Example Implimentation
In this section we will extend the ThreadPool concept. We want a progress bar to visualize how much of the compute has finished at any given point in time. We also want to automagically capture the exceptions (if the user desires) so that the program does not hault and a graceful exit can be orchestrated.

In [21]:
import time
import multiprocessing
import multiprocessing.pool
from tqdm.notebook import tqdm

# This class will keep track of exceptions raised in multitprocess functions if the user
# has elected to do so when they create the EnhancedThreadPool

class EnhancedProcessPoolException(Exception):
    
    def __init__(self, message):
        super().__init__(message)
    def __init__(self, function, parameters, root_exeption, message):
        self.functiom=function
        self.parameters=parameters
        self.__cause__=root_exeption
        self.message = message

# This class will act as an enhanced threadpool
# Ideally i would extend the ThreadPool class but I do not have enough time to impliment 
# ALL the required functionality. As such it will remain a separate class until I can extend.

class EnhancedProcessPool():

    def __init__(self, num_workers=4, handle_exceptions=True, shared_memory="epp"):
        
        self.handle_exceptions = handle_exceptions
        
        # Create a pointer for a progress bar to track progress
        self.progress_bar = None
        
        # Create a lock to manage internal access to the progress bar
        self.manager = multiprocessing.Manager()
        self.lock = self.manager.Lock()
        
        # Create internal pool to manage processes
        self.pool = multiprocessing.Pool(processes=num_workers)
        
        # Create a shared memory object to store the number of completed threads
        # between the various threads running in parallel
        self.shared_memory_name = shared_memory
        try:
            self.shared_memory = multiprocessing.shared_memory.SharedMemory(name=self.shared_memory_name, create=True, size=1)
        except FileExistsError as fee:
            self.shared_memory = multiprocessing.shared_memory.SharedMemory(name=self.shared_memory_name, create=False, size=1)
        self.shared_memory_buffer = self.shared_memory.buf
        self.shared_memory_buffer[0] = 0

    def _create_progress_bar(self, num_ops):
        
        return tqdm(range(num_ops))
        
    def _update_progress_bar(self, i):
        
        self.progress_bar.last_print_n = i
        self.progress_bar.n = i
        self.progress_bar.set_description(f'Completed Ops')
        self.progress_bar.refresh()

    # To make things simple we will define an internal function to hide the details
    # of the progress bar. This way the user can wwrite functions without worrying
    # about updating or locking the progress bar.
    @staticmethod
    def _wrapper_function(func, args, kwargs, internal_params):
        
        # Extract internal arguments for the manager's internal functions
        internal_args, internal_kwargs = internal_params
        lock, shared_memory_name = internal_args
        handle_exceptions = internal_kwargs["handle_exceptions"]
        
        # Create pointer to shared memory (so we can update the progress bar)
        sm = multiprocessing.shared_memory.SharedMemory(name=shared_memory_name, create=False, size=1)
        smb = sm.buf
        
        # Run the user specificed function and return the results
        # Handle the exceptions is the user has requested us to do so
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if handle_exceptions:
                return EnhancedProcessPoolException(func, (args,kwargs), e, "An error ocurred while running the parallel function")
            else:
                raise
        finally:
            # Update the buffer for the progress bar to indicate the function has completed
            lock.acquire()
            smb[0] = int(smb[0]) + 1
            lock.release()
            sm.close()
        
    # This function will run a set of functions in separate processes.
    # The main thread will block and update a progress bar as the functions complete
    # It will return a result set once all the processes have finished or an
    # error has been encountered.
    # This function will run a set of functions in separate processes.
    # The main thread will block and update a progress bar as the functions complete
    # It will return a result set once all the processes have finished or an
    # error has been encountered.
    def parralelize(self, func, arg_set, kwarg_set):
        
        # Setup vars to help kick off parallelization
        num_ops = len(arg_set)
        self.progress_bar = self._create_progress_bar(num_ops)
        self.shared_memory_buffer[0] = 0
        
        # Configure the parameters for the parallelization
        func_set = [func for i in range(0, num_ops)]
        internal_param_set = [
            ([self.lock, self.shared_memory_name], 
             {"handle_exceptions": self.handle_exceptions}
            ) for i in range(0, num_ops)]   
        param_set = zip(func_set, arg_set, kwarg_set, internal_param_set)
        
        # Start the functions in separate parallel processes and don't block
        apply_result = self.pool.starmap_async(self._wrapper_function, param_set)
        
        # Wait for the proceses to finish while updating the progress bar
        while not apply_result.ready():
            self._update_progress_bar(self.shared_memory_buffer[0])
            time.sleep(0.5)
        self._update_progress_bar(self.shared_memory_buffer[0])
        
        # Cleanup
        self.shared_memory.close()
        self.shared_memory.unlink()

        # Return the results
        return apply_result.get()

Now that we have these base objects we can test them out.

In [23]:
import itertools

# Define the function we want to parallelize
def my_func(idx, lock):
    if idx == 4:
        raise Exception("This should have been caught")
    for i in range(0, 3):
        lock.acquire()
        print("{0} -> {1}".format(idx, i))
        lock.release()
        time.sleep(1)
    return idx

# Detine parameters for multithreading
num_ops = 10
m = multiprocessing.Manager()
lock = m.Lock()
my_func_arg_set = list(zip(list(range(0, num_ops)), itertools.repeat(lock)))
my_func_kwarg_set = [{} for i in range(0, num_ops)]

# Start the process in parallel
epp = EnhancedProcessPool(num_workers=4)
results = epp.parralelize(my_func, my_func_arg_set, my_func_kwarg_set)
print("")
print(results)

  0%|          | 0/10 [00:00<?, ?it/s]

0 -> 0
1 -> 0
3 -> 0
2 -> 0
0 -> 1
1 -> 1
3 -> 1
2 -> 1
0 -> 2
1 -> 2
3 -> 2
2 -> 2
6 -> 0
5 -> 0
7 -> 0
8 -> 0
6 -> 15 -> 1

7 -> 1
8 -> 1
6 -> 2
5 -> 2
7 -> 2
8 -> 2
9 -> 0
9 -> 1
9 -> 2

[0, 1, 2, 3, EnhancedProcessPoolException(<function my_func at 0x7fbc546748b0>, ((4, <AcquirerProxy object, typeid 'Lock' at 0x7fbc7006bc40>), {}), Exception('This should have been caught'), 'An error ocurred while running the parallel function'), 5, 6, 7, 8, 9]
